# CORONAVIRUS CAPITAL PROJECTS FUND 9.8 billion

https://home.treasury.gov/policy-issues/coronavirus/assistance-for-state-local-and-tribal-governments/capital-projects-fund

https://home.treasury.gov/system/files/136/Allocations-Methodology-States-Territories-Freely-Associated-States.pdf


## 52 States (including DC and Puerto Rico)





In [15]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from datetime import datetime

import geopandas as gp
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from shapely.geometry import Point
from adjustText import adjust_text

from elasticsearch import Elasticsearch, helpers
import json

# PARAMS

In [16]:
STATE_FIPS_DICT_52 = {
'01': 'AL', 
'02': 'AK',
'04': 'AZ',
'05': 'AR',
'06': 'CA',
'08': 'CO',
'09': 'CT',
'10': 'DE',
'11': 'DC',
'12': 'FL',
'13': 'GA',
'15': 'HI',
'16': 'ID',
'17': 'IL',
'18': 'IN',
'19': 'IA',
'20': 'KS',
'21': 'KY',
'22': 'LA',
'23': 'ME',
'24': 'MD',
'25': 'MA',
'26': 'MI',
'27': 'MN',
'28': 'MS',
'29': 'MO',
'30': 'MT',
'31': 'NE',
'32': 'NV',
'33': 'NH',
'34': 'NJ',
'35': 'NM',
'36': 'NY',
'37': 'NC',
'38': 'ND',
'39': 'OH',
'40': 'OK',
'41': 'OR',
'42': 'PA',
'72': 'PR',
'44': 'RI',
'45': 'SC',
'46': 'SD',
'47': 'TN',
'48': 'TX',
'49': 'UT',
'50': 'VT',
'51': 'VA',
'53': 'WA',
'54': 'WV',
'55': 'WI',
'56': 'WY'}


STATE_TO_ABBREV_52 = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "Puerto Rico": "PR",
}   

STATE_FIPS_DICT_5 = {'60': "AS", '66': "GU", '69': "MP", '78': "VI", 'NoFIPS': "UM"}
STATE_TO_ABBREV_5 =     { 
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "U.S. Virgin Islands": "VI", 
    "United States Minor Outlying Islands": "UM",}

STATE_ABBRV_LIST_52 = list(sorted(STATE_FIPS_DICT_52.values()))

In [17]:
# NOTE
# FORMAT of DICTIONARY
# e.g. { 'AL': { 'population' : X, 'rural': Y, 'rural_margin_error': Z, 'poverty150': K, 'poverty150_margin_error': M}, ... } 
ALL_STATES_DICT = {} 

for state_abbrev in STATE_ABBRV_LIST_52:
    ALL_STATES_DICT[state_abbrev] = {}


# 1. Population

In [19]:
# 2020 decennial census: https://www.census.gov/data/tables/2020/dec/2020-apportionment-data.html

# Table 1. Apportionment Population and Number of Representatives by State: 2020 Census
# ONLY HAS 50 STATES: The populations of the District of Columbia and Puerto Rico are not included in the apportionment population 
# because they do not have voting seats in the U.S. House of Representatives.
STATE_POPULATION_2020 = pd.read_excel('apportionment-2020-table01.xlsx', header=3, index_col=0)  
for i, row in STATE_POPULATION_2020.iterrows():
    # print(i, 'row', row[0])
    if i in STATE_TO_ABBREV_52:
        ALL_STATES_DICT[STATE_TO_ABBREV_52[i]]["population"] = row[0]
        
# Table 2. Resident Population for the 50 States, the District of Columbia, and Puerto Rico: 2020 Census
# POPULATION DC "District of Columbia" and PR "Puerto Rico"
DC_PR_POPULATION_2020 = pd.read_excel('apportionment-2020-table02.xlsx', header=3, index_col=0)
for i, row in DC_PR_POPULATION_2020.iterrows():
    if i == "District of Columbia" or i == "Puerto Rico":
        ALL_STATES_DICT[STATE_TO_ABBREV_52[i]]["population"] = row[0]
        

# 2. Rural

https://www.census.gov/content/dam/Census/library/publications/2020/acs/acs_general_handbook_2020_ch02.pdf

Census tracts are small geographic areas—with an
average of about 4,000 people each—that are commonly used to present information for small towns,
rural areas, and neighborhoods.


In [20]:
RURAL_POPULATION = pd.read_csv('rural/ACSDT1Y2019.B01003_data_with_overlays_2021-10-11T163857.csv', header=0, index_col=0)

# NOTE: No rural population for DC 
ALL_STATES_DICT['DC']['rural'] = 0
ALL_STATES_DICT['DC']['rural_margin_error'] = 0

for i, row in RURAL_POPULATION.iterrows():
    statefips = i[-2:]
    if statefips in STATE_FIPS_DICT_52:
        state_abbrv = STATE_FIPS_DICT_52[statefips]
        ALL_STATES_DICT[state_abbrv]["rural"] = float(row[1])
        ALL_STATES_DICT[state_abbrv]["rural_margin_error"] = float(row[2])


# 3. Poverty line

In [21]:
filename = 'poverty/ACSST1Y2019.S1701_2021-10-12T115526/ACSST1Y2019.S1701_data_with_overlays_2021-10-12T113421.csv'
POVERTY_POPULATION = pd.read_csv(filename, header=0, index_col=0)
# NOTE: No rural population for DC 

col_estimate = 'S1701_C01_040E'
col_margin_error = 'S1701_C01_040M'
POVERTY_POPULATION = POVERTY_POPULATION[[col_estimate, col_margin_error]]
POVERTY_POPULATION = POVERTY_POPULATION.rename(columns = {'S1701_C01_040E':'total', 'S1701_C01_040M':'margin_error'})
POVERTY_POPULATION.head(3)

,total,margin_error
GEO_ID,,
id,Estimate!!Total!!Population for whom poverty s...,Margin of Error!!Total!!Population for whom po...
0400000US01,1214017,29293
0400000US02,125335,8411


In [22]:
# NOTE:
for i, row in POVERTY_POPULATION.iterrows():
    statefips = i[-2:]
    if statefips in STATE_FIPS_DICT_52:
        state_abbrv = STATE_FIPS_DICT_52[statefips]
        ALL_STATES_DICT[state_abbrv]["poverty150"] = float(row[0])
        ALL_STATES_DICT[state_abbrv]["poverty150_margin_error"] = float(row[1])


# Calculate fund allocation for each state

ALL_STATES_DICT FORMAT: 

{ 'AL': { 'population' : X, 'rural': Y, 'rural_margin_error': Z, 'poverty150': K, 'poverty150_margin_error': M}, ... } 

In [23]:
# ALLOCATION AMOUNTS, CURRENCY UNIT = million USD
FIXED_BASE_PER_STATE = 100.0 # 100 million
POPULATION_ALLOCATION_TOTAL = 2300 # 2300 million = 2.3 billion
RURAL_ALLOCATION_TOTAL = 1150 
POVERTY_ALLOCATION_TOTAL = 1150


US_POPULATION = sum(ALL_STATES_DICT[state]['population'] for state in STATE_ABBRV_LIST_52)

US_RURAL = sum(ALL_STATES_DICT[state]['rural'] for state in STATE_ABBRV_LIST_52)
US_RURAL_MARGIN_ERROR = sum(ALL_STATES_DICT[state]['rural_margin_error'] for state in STATE_ABBRV_LIST_52)

US_POVERTY150 = sum(ALL_STATES_DICT[state]['poverty150'] for state in STATE_ABBRV_LIST_52)
US_POVERTY150_MARGIN_ERROR = sum(ALL_STATES_DICT[state]['poverty150_margin_error'] for state in STATE_ABBRV_LIST_52)

print("total population ", US_POPULATION, "rural", US_RURAL, "+/-", US_RURAL_MARGIN_ERROR, "poverty150", US_POVERTY150, "+/-", US_POVERTY150_MARGIN_ERROR)


for state in STATE_ABBRV_LIST_52:
    # Start with a fixed base
    ALL_STATES_DICT[state]['allocation'] = FIXED_BASE_PER_STATE
    # population
    ALL_STATES_DICT[state]['allocation'] += ALL_STATES_DICT[state]['population']/US_POPULATION * POPULATION_ALLOCATION_TOTAL
    # rural
    ALL_STATES_DICT[state]['allocation'] += ALL_STATES_DICT[state]['rural']/US_RURAL * RURAL_ALLOCATION_TOTAL
    # poverty150
    ALL_STATES_DICT[state]['allocation'] += ALL_STATES_DICT[state]['poverty150']/US_POVERTY150 * POVERTY_ALLOCATION_TOTAL
    
    
print("TOTAL ALLOCATION should be 9800 million ", sum(ALL_STATES_DICT[state]['allocation'] for state in STATE_ABBRV_LIST_52))

for full_state_name in STATE_TO_ABBREV_52:
    state = state = STATE_TO_ABBREV_52[full_state_name]
    amount = '${:,.2f}'.format(ALL_STATES_DICT[state]['allocation'] * 1000000)
    ALL_STATES_DICT[state]['allocation_currency'] = amount
    

    

total population  335083853.0 rural 63931752.0 +/- 990233.0 poverty150 67044983.0 +/- 1387076.0
TOTAL ALLOCATION should be 9800 million  9800.0


## Compare with the actual allocated amounts

In [24]:
from re import sub
from decimal import Decimal

In [25]:
ACTUAL_ALLOCATIONS = pd.read_csv('Allocations-States.csv', header=4, index_col=0)
# NOTE the indexing by integer iloc [row_indexer, column_indexer]
ACTUAL_ALLOCATIONS = ACTUAL_ALLOCATIONS.iloc[:,:2]


for full_state_name, row in ACTUAL_ALLOCATIONS.iterrows():
    if full_state_name in STATE_TO_ABBREV_52:
        state = STATE_TO_ABBREV_52[full_state_name]
        ACTUAL_ALLOCATIONS.loc[full_state_name, "Ready’s Projected Allocation"] = ALL_STATES_DICT[state]['allocation_currency']
        
        projected = ALL_STATES_DICT[state]['allocation'] * 1000000
        # https://stackoverflow.com/a/8422055
        actual = float(sub(r'[^\d.]', '', row['ACTUAL Allocation']))
        ACTUAL_ALLOCATIONS.loc[full_state_name, "Error Percentage (%)"] = (abs(projected-actual)/actual) * 100

        
ACTUAL_ALLOCATIONS.to_csv('completed-calculation.csv')

### Experiment w/ censusdata : find relevant ACS table

In [212]:
import censusdata
import re

In [262]:
censusdata.download('acs5', 2019,  censusdata.censusgeo([('state', '*')]), ['B07412_2'])

ValueError: Unexpected response (URL: https://api.census.gov/data/2019/acs/acs5?get=NAME,B07412_2&for=state:*): error: error: unknown variable 'B07412_2' 

In [249]:
censusdata.search('acs5', 2019, 'concept', 'Population Poverty Status')

[]

# IIJA 42.5 billion (for states) /$65 billion 

https://www.brookings.edu/blog/the-avenue/2021/08/13/the-senate-infrastructure-bills-four-interconnected-broadband-components/

In [26]:
# 42.5 billion in total; CURRENCY UNIT = million USD
IIJA_STATE_TOTAL = 42500 

HIGH_COST_ALLOCATION_TOTAL = 4250 # 10% i.e. 4.25 billion
FIXED_BASE_PER_STATE = 100.0 
OTHER_TERRITORIES_ALLOCATION = 100.0

REMAIN_TOTAL = IIJA_STATE_TOTAL - HIGH_COST_ALLOCATION_TOTAL - OTHER_TERRITORIES_ALLOCATION - FIXED_BASE_PER_STATE * 52

POPULATION_ALLOCATION_TOTAL = REMAIN_TOTAL / 2
RURAL_ALLOCATION_TOTAL = REMAIN_TOTAL / 4
POVERTY_ALLOCATION_TOTAL = REMAIN_TOTAL / 4


In [30]:
for state in STATE_ABBRV_LIST_52:
    # Start with a fixed base
    ALL_STATES_DICT[state]['allocation'] = FIXED_BASE_PER_STATE
    # population
    ALL_STATES_DICT[state]['allocation'] += ALL_STATES_DICT[state]['population']/US_POPULATION * POPULATION_ALLOCATION_TOTAL
    # rural
    ALL_STATES_DICT[state]['allocation'] += ALL_STATES_DICT[state]['rural']/US_RURAL * RURAL_ALLOCATION_TOTAL
    # poverty150
    ALL_STATES_DICT[state]['allocation'] += ALL_STATES_DICT[state]['poverty150']/US_POVERTY150 * POVERTY_ALLOCATION_TOTAL


for_52_states_allocation = IIJA_STATE_TOTAL - HIGH_COST_ALLOCATION_TOTAL - OTHER_TERRITORIES_ALLOCATION
print(f"TOTAL ALLOCATION for 52 states should be: {for_52_states_allocation}, v.s. projected calculation: {sum(ALL_STATES_DICT[state]['allocation'] for state in STATE_ABBRV_LIST_52)}")

for full_state_name in STATE_TO_ABBREV_52:
    state = state = STATE_TO_ABBREV_52[full_state_name]
    amount = '${:,.2f}'.format(ALL_STATES_DICT[state]['allocation'] * 1000000)
    ALL_STATES_DICT[state]['allocation_currency'] = amount
    


TOTAL ALLOCATION for 52 states should be: 38150.0, v.s. projected calculation: 38149.99999999999


In [31]:
REVERSED_STATE_FIPS_DICT_52 = {v: k for k, v in STATE_FIPS_DICT_52.items()}

IIJA_ALLOCATION = []

for full_state_name in sorted(STATE_TO_ABBREV_52.keys()):
    state = STATE_TO_ABBREV_52[full_state_name]
    amount = ALL_STATES_DICT[state]['allocation'] * 1000000
    amount_currency = ALL_STATES_DICT[state]['allocation_currency']
    IIJA_ALLOCATION.append([full_state_name, state, 
                            REVERSED_STATE_FIPS_DICT_52[state], 
                             amount, amount_currency])

IIJA_ALLOCATION = pd.DataFrame(IIJA_ALLOCATION, columns=['STATE', 'ABBRV', 'STATEFIPS', 'AMOUNT', 'AMOUNT (human-readable)'])

IIJA_ALLOCATION.to_csv('iija-completed-projection.csv')